# Exercise 7: Visual Advantage Actor-Critic (A2C)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.python.framework import ops

from unityagents import UnityEnvironment

%matplotlib inline

### Hyperparameters

In [ ]:
y = 0.9 # Discount rate.
total_episodes = 10000 # Set total number of episodes to train agent on.
update_frequency = 20 # How many episodes before updating model.
learning_rate = 3e-4 # Agent learning rate.
summary_freq = 50
hidden_units = 128 # Number of units in hidden layer.
summary_freq = 100 # How often to display summary statistics.
model_path = "./models/vac" # The path to save our model to.
summary_path = "./summaries/vac" # The path to save our model to.
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.

### Load the environment

In [ ]:
env = UnityEnvironment("./envs/GridWorld", worker_id=6)
default_brain = env.brain_names[0]

### Examine observation space

In [ ]:
# Reset the environment
env_info = env.reset()[default_brain]

# Examine the state space for the default brain
print("Agent state looks like: \n{}".format(env_info.vector_observations[0]))

# Examine the observation space for the default brain
for observation in env_info.visual_observations:
    print("Agent observations look like:")
    if observation.shape[3] == 3:
        plt.imshow(observation[0,:,:,:])
    else:
        plt.imshow(observation[0,:,:,0])

### The Actor Critic Agent

In [ ]:
class ACAgent():
    def __init__(self, lr, a_size, h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.observation_in =tf.placeholder(shape=[None, 32, 32, 3],dtype=tf.float32)
        self.conv1 = tf.layers.conv2d(self.observation_in, 16, 
                                 kernel_size=[3, 3], strides=[2, 2], 
                                 use_bias=False,
                                 activation=tf.nn.elu)
        self.conv2 = tf.layers.conv2d(self.conv1, 32, 
                                 use_bias=False,
                                 kernel_size=[3, 3], 
                                 strides=[2, 2], 
                                 activation=tf.nn.elu)
        self.batch_size = tf.placeholder(shape=None, dtype=tf.int32)
        hidden = tf.layers.dense(tf.layers.flatten(self.conv2),h_size,use_bias=False,activation=tf.nn.elu)
        self.policy = tf.layers.dense(hidden,a_size,activation=tf.nn.softmax,use_bias=False)
        self.value = tf.layers.dense(hidden,1,activation=None,use_bias=False)
        self.entropy = -tf.reduce_sum(self.policy * tf.log(self.policy + 1e-10), axis=1)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions = tf.one_hot(self.action_holder, a_size)
        
        self.taken_action = tf.reduce_sum(self.policy * self.actions, axis=1)
        advantage = self.reward_holder - tf.stop_gradient(tf.reduce_sum(self.value, axis=1))
        mean_adv, var_adv = tf.nn.moments(advantage, axes=0)
        normalized_advantage = (advantage - mean_adv) / (tf.sqrt(var_adv) + 1e-10)
        
        self.policy_loss = -tf.reduce_mean(tf.log(self.taken_action + 1e-10) * normalized_advantage)
        self.value_loss = tf.reduce_mean(tf.squared_difference(self.reward_holder, tf.reduce_sum(self.value,axis=1)))
        self.loss = self.policy_loss + 0.5 * self.value_loss - 0.01 * self.entropy
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss,tvars)
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

### Training the Agent

In [ ]:
# Function to take list of rewards and discount factor
# and produce discounted sum of future rewards.
def discount_rewards(r, gamma):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [ ]:
if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(summary_path):
    os.makedirs(summary_path)

# Load the agent.
action_size = env.brains[default_brain].vector_action_space_size
myAgent = ACAgent(lr=learning_rate, a_size=action_size, h_size=hidden_units)

# Start an interactive TensorFlow session.
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)
summary_writer = tf.summary.FileWriter(summary_path)
saver = tf.train.Saver()

# Optionally load the model.
if load_model == True:
    print('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_path)
    saver.restore(sess,ckpt.model_checkpoint_path)


# Create variables which will be used throughout training.
total_steps = 0
total_reward = []
total_length = []
total_entropy = []
total_value = []
v_losses = []
p_losses = []

# Create buffer for storing gradients.
gradBuffer = sess.run(tf.trainable_variables())
for ix,grad in enumerate(gradBuffer):
    gradBuffer[ix] = grad * 0

# Start training loop
for i in range(total_episodes):
    done = False
    brain_info = env.reset()[default_brain]
    observation = brain_info.visual_observations[0][0]
    running_reward = 0
    episode_steps = 0
    ep_history = []
    entropy = []
    while not done:
        episode_steps += 1
        total_steps += 1
        # Probabilistically pick an action given our network outputs.
        a_dist, ent, value = sess.run([myAgent.policy, myAgent.entropy, myAgent.value],
                          feed_dict={myAgent.observation_in:[observation], myAgent.batch_size: 1})
        a = np.random.choice(a_dist[0],p=a_dist[0])
        action = np.argmax(a_dist == a)
        brain_info = env.step(action)[default_brain]
        observation_1 = brain_info.visual_observations[0][0]
        reward = brain_info.rewards[0]
        done = brain_info.local_done[0]
        ep_history.append([observation, action, reward, observation_1, value])
        observation = observation_1
        running_reward += reward
        entropy.append(ent)
        if done and train_model:
            # Update the network.
            ep_history = np.array(ep_history)
            ep_history[:,2] = discount_rewards(ep_history[:,2], y)
            feed_dict={myAgent.reward_holder:ep_history[:,2],
                    myAgent.action_holder:ep_history[:,1],
                    myAgent.observation_in:np.stack(ep_history[:,0]),
                    myAgent.batch_size: len(ep_history)}
            v_loss, p_loss, grads = sess.run([myAgent.value_loss, 
                                              myAgent.policy_loss, 
                                              myAgent.gradients], 
                                             feed_dict=feed_dict)
            v_losses.append(v_loss)
            p_losses.append(p_loss)
            for idx,grad in enumerate(grads):
                gradBuffer[idx] += grad

            if i % update_frequency == 0 and i != 0:
                feed_dict = dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0

    total_reward.append(running_reward)
    total_length.append(episode_steps)
    total_entropy.append(np.mean(entropy))
    total_value.append(np.mean(ep_history[:,4]))

    # Write training statistics to tensorboard.
    if i % summary_freq == 0 and i != 0:
        summary = tf.Summary()
        summary.value.add(tag='Info/Mean Reward', simple_value=float(np.mean(total_reward[-summary_freq:])))
        summary.value.add(tag='Info/Episode Length', simple_value=float(np.mean(total_length[-summary_freq:])))
        summary.value.add(tag='Info/Value Estimate', simple_value=float(np.mean(total_value[-summary_freq:])))
        summary.value.add(tag='Info/Value Loss', simple_value=float(np.mean(v_losses[-summary_freq:])))
        summary.value.add(tag='Info/Policy Loss', simple_value=float(np.mean(p_losses[-summary_freq:])))
        summary.value.add(tag='Info/Entropy', simple_value=float(np.mean(total_entropy[-summary_freq:])))
        summary_writer.add_summary(summary, i)
        summary_writer.flush()
        print("Step: {}, Episode: {}, Mean Reward: {}".format(str(total_steps), str(i),
                                                              str(round(np.mean(total_reward[-summary_freq:]), 3))))        
    # Save agent's model
    if i % 1000 == 0 and i != 0:
        saver.save(sess, model_path+'/model-'+str(i)+'.cptk')
        print("Saved Model")
env.close()

In [ ]:
env.close()